In [41]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [42]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [43]:
# load train and test dataset
def load_dataset(isCategorical=True):
    # load dataset
    data_x = np.load('data_x_aumentated_normalized.npy')
    data_y = np.load('data_y_aumentated_normalized.npy')
    #data_x = np.load('data_x_original_augmented.npy')
    #data_y = np.load('data_y_original_augmented.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
    if isCategorical == True:
        # one hot encode target values
        train_y = to_categorical(train_y)
        test_y = to_categorical(test_y)
        validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [44]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [45]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset(False)
#trainX, testX = prep_pixels(trainX, testX)

TOTAL_IMAGES =  2368
train_size =  1894
test_size =  236
validation_size =  238


In [46]:
clf1 = tf.keras.models.load_model("displasys_mobilenet-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [47]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [48]:
clf2 = tf.keras.models.load_model("displasys_resnet50v2-baseline-holdout-100epochs_dataAugmenting_l2_dropout.npy")

In [49]:
clf3 = tf.keras.models.load_model("resnet50-baseline-holdout-100epochs_normalized.npy")

# VGG + Flatting + Gaussian

In [50]:
clf1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          294944    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                        

In [51]:
vgg_lr_flatting = keras.Model(inputs=clf1.input, outputs=clf1.get_layer(index=9).output)
vgg_featureVector_flatting = vgg_lr_flatting.predict(trainX)

60/60 [==============================] - 2s 30ms/step


In [52]:
vgg_featureVector_flatting.shape

(1894, 64)

In [53]:
trainY.shape

(1894,)

In [54]:
VGG_LR_FLATTING = GaussianNB().fit(vgg_featureVector_flatting, np.ravel(trainY))

In [55]:
vgg_featureVector_flatting_test = vgg_lr_flatting.predict(testX)

8/8 [==============================] - 0s 29ms/step


In [56]:
acc = VGG_LR_FLATTING.score(vgg_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 87.288


# MobileNet + Flatting + LogisticRegression

In [57]:
clf2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          589856    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 32)         0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 64)          1

In [58]:
mobile_lr_flatting = keras.Model(inputs=clf2.input, outputs=clf2.get_layer(index=9).output)
mobile_featureVector_flatting = mobile_lr_flatting.predict(trainX)

60/60 [==============================] - 6s 81ms/step


In [59]:
mobile_featureVector_flatting.shape

(1894, 64)

In [60]:
mobile_LR_FLATTING = RandomForestClassifier(n_estimators=100, random_state=1).fit(mobile_featureVector_flatting, np.ravel(trainY))

In [61]:
mobile_featureVector_flatting_test = mobile_lr_flatting.predict(testX)

8/8 [==============================] - 1s 158ms/step


In [62]:
acc = mobile_LR_FLATTING.score(mobile_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 94.915


# Resnet + Flatting + LogisticRegression

In [63]:
clf3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 4, 4, 32)          589856    
                                                                 
 batch_normalization (BatchN  (None, 4, 4, 32)         128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4, 4, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 64)          1

In [64]:
resnet_lr_flatting = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=9).output)
resnet_featureVector_flatting = resnet_lr_flatting.predict(trainX)

60/60 [==============================] - 6s 78ms/step


In [65]:
resnet_featureVector_flatting.shape

(1894, 64)

In [66]:
resnet_LR_FLATTING = LogisticRegression(random_state=0).fit(resnet_featureVector_flatting, np.ravel(trainY))

In [67]:
resnet_featureVector_flatting_test = resnet_lr_flatting.predict(testX)

8/8 [==============================] - 1s 103ms/step


In [68]:
acc = resnet_LR_FLATTING.score(resnet_featureVector_flatting_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 96.186


# Resnet + Pooling + LogisticRegression

In [69]:
#new_model_lr_pooling = keras.Model(inputs=clf3.input, outputs=clf3.get_layer(index=8).output)
#featureVector_pooling = new_model_lr_pooling.predict(trainX)

In [70]:
#featureVector_pooling.shape

In [71]:
#LR_POOLING = LogisticRegression(random_state=0).fit(featureVector_pooling, np.ravel(trainY))

In [72]:
#featureVector_pooling_test = new_model_lr_pooling.predict(testX)

In [73]:
#acc = LR_POOLING.score(featureVector_pooling_test, testY)
#print('Acurácia: %.3f' % (acc * 100.0))

# Emsemble Gaussian + RandomForest + LogisticRegression (VGG , MobileNet, Resnet)

In [74]:
#creating the emsemble
eclf = VotingClassifier(estimators=[('lr_vgg', VGG_LR_FLATTING), ('gaussian_mobile', mobile_LR_FLATTING), ('rf_resnet', resnet_LR_FLATTING)], voting='soft')

In [75]:
#joining the features obtained by flatting on all models
emsemble_feature_vector = np.append(vgg_featureVector_flatting, mobile_featureVector_flatting, axis=1)
emsemble_feature_vector = np.append(emsemble_feature_vector, resnet_featureVector_flatting, axis=1)

In [76]:
emsemble_feature_vector.shape

(1894, 192)

In [77]:
#training the emsemble with all features
eclf.fit(emsemble_feature_vector, np.ravel(trainY))

VotingClassifier(estimators=[('lr_vgg', GaussianNB()),
                             ('gaussian_mobile',
                              RandomForestClassifier(random_state=1)),
                             ('rf_resnet', LogisticRegression(random_state=0))],
                 voting='soft')

In [78]:
emsemble_feature_vector_test = np.append(vgg_featureVector_flatting_test, mobile_featureVector_flatting_test, axis=1)
emsemble_feature_vector_test = np.append(emsemble_feature_vector_test, resnet_featureVector_flatting_test, axis=1)

In [79]:
acc = eclf.score(emsemble_feature_vector_test, testY)
print('Acurácia: %.3f' % (acc * 100.0))

Acurácia: 95.339
